In [1]:
import psycopg2
import csv
import glob
import numpy as np 
import pandas as pd
from scipy import stats
from sqlalchemy import create_engine

In [2]:
#create the database

# ! createdb mod2project


In [3]:
#connect to the database and make a cursor
conn = psycopg2.connect("dbname=mod2project")
cur = conn.cursor()

In [4]:
#set up our SQL tables
create_query_1 = """
CREATE TABLE course_offerings(
  uuid text
, course_uuid text 
, term_code integer
, name text
)
"""

create_query_2 = """
CREATE TABLE courses(
  uuid text 
, name text 
, number integer
)
"""

create_query_3 = """
CREATE TABLE grade_distributions(
  course_offering_uuid text
, section_number text 
, a_count integer
, ab_count integer
, b_count integer
, bc_count integer
, c_count integer
, d_count integer
, f_count integer
, s_count integer
, u_count integer
, cr_count integer
, n_count integer
, p_count integer
, i_count integer
, nw_count integer
, nr_count integer
, other_count integer
)
"""

create_query_4 = """
CREATE TABLE instructors(
  id integer
, name text
)
"""

create_query_5 = """
CREATE TABLE rooms(
  uuid text
, facility_code text
, room_code text
)
"""

create_query_6 ="""
CREATE TABLE schedules(
  uuid text
, start_time integer
, end_time integer
, mon text
, tues text
, wed text
, thurs text
, fri text
, sat text
, sun text
)
"""

create_query_7 = """
CREATE TABLE sections(
  uuid text
, course_offering_uuid text
, section_type text
, number text
, room_uuid text
,schedule_uuid text
)
"""

create_query_8 = """
CREATE TABLE subject_memberships(
  subject_code integer
, course_offering_uuid text
)
"""

create_query_9 = """
CREATE TABLE subjects(
  code text
, name text
, abbreviation text
)
"""

create_query_10 = """
CREATE TABLE teachings(
  instructor_id integer
, section_uuid text
)
"""

In [5]:
#get the names and locations of our datafiles
csvs_for_table = glob.glob('raw_data/*.csv')

In [6]:
#make a list of our queries
queries_list = [create_query_1,create_query_2,create_query_3,create_query_4
               ,create_query_5,create_query_6,create_query_7
               ,create_query_8,create_query_9,create_query_10]

In [7]:
#extract table names from our file names
table_names = [x.split('/')[1][:-4] for x in csvs_for_table]

In [8]:
#loop through our queries list and execute them

# for query in queries_list:
#     cur.execute(query)
#     conn.commit()

In [9]:
# #loop through our table names and indexes
# for q_num, table_name in enumerate(table_names):
    
#     #extract values from our first queries to use
#     #in our insert statements below
#     fields = tuple(line.strip(", ").split()[0] for 
#                    line in queries_list[q_num].split('\n')[2:-2])
    
#     #set up our insert statements
#     query = f'''
#     INSERT INTO {table_name}
#     ({', '.join(fields)})
#     VALUES ({', '.join('%s' for _ in range(len(fields)))})
#     '''
        
    
#     #open each file and insert it's values row
#     #by row into the appropriate tables
#     with open(csvs_for_table[q_num], 'r') as file:
#         reader = csv.reader(file)
#         next(reader)  # Skip the header row.
#         for row in reader:
#             cur.execute(
#                 query,
#                 row
#             )
#             conn.commit()
            
#         #check where we're having problems
#         print(table_name,'still good')

In [10]:
# conn.rollback()


In [11]:
#set up a sqlalchemy engine

engine = create_engine("postgresql:///mod2project", echo=True)


In [38]:
#build our query
q = """
SELECT 
  sub_m.subject_code
, sub_j.name AS sub_j_name
, grade_d.a_count
, grade_d.ab_count
, grade_d.b_count
, grade_d.bc_count
, grade_d.c_count
, sch.start_time
, sch.end_time
,

  CASE
        WHEN term_code BETWEEN 1063 AND 1073 THEN '2006'
        WHEN term_code BETWEEN 1073 AND 1083 THEN '2007'
        WHEN term_code BETWEEN 1083 AND 1093 THEN '2008'
        WHEN term_code BETWEEN 1093 AND 1103 THEN '2009'
        WHEN term_code BETWEEN 1103 AND 1113 THEN '2010'
        WHEN term_code BETWEEN 1113 AND 1123 THEN '2011'
        WHEN term_code BETWEEN 1123 AND 1133 THEN '2012'
        WHEN term_code BETWEEN 1133 AND 1143 THEN '2013'
        WHEN term_code BETWEEN 1143 AND 1153 THEN '2014'
        WHEN term_code BETWEEN 1153 AND 1163 THEN '2015'
        WHEN term_code BETWEEN 1163 AND 1173 THEN '2016'
        WHEN term_code BETWEEN 1173 AND 1183 THEN '2017'
        ELSE 'the_world_is_over'
    END AS Year
    

FROM subject_memberships AS sub_m
JOIN subjects sub_j
ON cast(sub_m.subject_code as text) = cast(sub_j.code as text)
JOIN course_offerings cor_o
ON cor_o.uuid = sub_m.course_offering_uuid
JOIN grade_distributions grade_d
ON grade_d.course_offering_uuid = cor_o.uuid
JOIN sections sect
ON sect.course_offering_uuid = cor_o.uuid
JOIN schedules sch
ON sect.schedule_uuid = sch.uuid

"""

In [39]:
#put our query into a dataframe
df_join = pd.read_sql_query(q,con=engine)

2019-05-09 08:59:08,585 INFO sqlalchemy.engine.base.Engine 
SELECT 
  sub_m.subject_code
, sub_j.name AS sub_j_name
, grade_d.a_count
, grade_d.ab_count
, grade_d.b_count
, grade_d.bc_count
, grade_d.c_count
, sch.start_time
, sch.end_time
,

  CASE
        WHEN term_code BETWEEN 1063 AND 1073 THEN '2006'
        WHEN term_code BETWEEN 1073 AND 1083 THEN '2007'
        WHEN term_code BETWEEN 1083 AND 1093 THEN '2008'
        WHEN term_code BETWEEN 1093 AND 1103 THEN '2009'
        WHEN term_code BETWEEN 1103 AND 1113 THEN '2010'
        WHEN term_code BETWEEN 1113 AND 1123 THEN '2011'
        WHEN term_code BETWEEN 1123 AND 1133 THEN '2012'
        WHEN term_code BETWEEN 1133 AND 1143 THEN '2013'
        WHEN term_code BETWEEN 1143 AND 1153 THEN '2014'
        WHEN term_code BETWEEN 1153 AND 1163 THEN '2015'
        WHEN term_code BETWEEN 1163 AND 1173 THEN '2016'
        WHEN term_code BETWEEN 1173 AND 1183 THEN '2017'
        ELSE 'the_world_is_over'
    END AS Year
    

FROM subjec

In [40]:
#create a unique set of course names
all_courses = set(df_join.sub_j_name)

In [41]:
#make a list of humanites
humanities = {'African Languages and Literature', 'Afro-American Studies', 
              'American Indian Studies', 'Art Department', 'Anthropology'
              'Art Education (Department of Art)', 'Art History', 'Asian American Studies',
              'Chicana/o and Latina/o Studies', 'Classics', 'Collaborative Nursing Program',
              'Communication Arts','Communication Sciences and Disorders', 'Community and Environmental Sociology',
              'Comparative Literature', 'Consumer Science', 'Counseling Psychology', 'Curriculum and Instruction',
              'Dance', 'Design Studies', 'ENGLISH', 'East Asian Area Studies', 'East Asian Languages and Literature',
              'Economics', 'Educational Leadership and Policy Analysis', 'Educational Policy Studies',
              'Educational Psychology', 'Emergency Medicine', 'Engineering Professional Development',
              'English', 'English as a Second Language', 'Entomology', 'Environmental Studies - Gaylord Nelson Institute',
              'FAMILY AND CONSUMER COMMUNICATIONS', 'Family Medicine', 'Farm & Industry Short Course',
              'Finance, Investment and Banking', 'Folklore Program', 'Forest and Wildlife Ecology',
              'French (French and Italian)', 'Gender and Women’s Studies', 'General Business',
              'Geography', 'German', 'Greek (Classics)', 'HEBREW', 'Hebrew-Biblical', 'Hebrew-Modern',
              'History', 'History of Science', 'Horticulture', 'Human Development and Family Studies',
              'Human Oncology', 'INDUSTRIAL RELATIONS', 'Integrated Arts', 'Integrated Liberal Studies',
              'Integrated Science', 'Interdisciplinary Courses (CALS)', 'Interdisciplinary Courses (L&S)',
              'Interdisciplinary Courses (SOHE)', 'International Business', 'International Studies',
              'Italian (French and Italian)', 'Jewish Studies', 'Journalism and Mass Communication',
              'Kinesiology', 'La Follette School of Public Affairs',  'Landscape Architecture',
              'Languages and Cultures of Asia', 'Languages and Cultures of Asia - Languages',
              'Latin (Classics)', 'Latin American, Caribbean, and Iberian Studies',
              'Law','Legal Studies','Library and Information Studies',
              'Life Sciences Communication', 'Linguistics', 'Literature in Translation',
              'Management and Human Resources', 'Marketing', 'Medical History and Bioethics',
              'Medieval Studies','Music','Music-Performance','Nursing','Nutritional Sciences',
              'Occupational Therapy (Department of Kinesiology)',
              'Operations and Technology Management','PHYSICAL EDUC ACTIVITY PROGM',
              'PROFESSIONAL ORIENTATION', 'Pharmacy','Pharmacy Practice','Philosophy',
              'Physical Therapy','Physician Assistant Program','Political Science',
              'Portuguese (Spanish and Portuguese)','Psychiatry',
              'Real Estate and Urban Land Economics',
              'Rehabilitation Psychology and Special Education',
              'Religious Studies','Risk Management and Insurance',
              'Scandinavian Studies','Slavic (Slavic Languages)','Social Work',
              'Social and Administrative Pharmacy','Sociology','Soil Science',
              'Spanish (Spanish and Portuguese)','TRANSPORTATION AND PUBLIC UTILITIES',
              'Theatre and Drama','Therapeutic Science (Department of Kinesiology)',
              'Urban and Regional Planning','WILDLIFE ECOLOGY', 'Zoology'
 
              }

In [42]:
#remove the humanites from the course list
all_courses -= humanities

In [43]:
#set a STEM
STEM = all_courses

In [44]:
#make columns with grades adjusted for weight
df_join['weighted_a'] = df_join.a_count * 0.95
df_join['weighted_ab'] = df_join.ab_count * 0.9
df_join['weighted_b'] = df_join.b_count * 0.85
df_join['weighted_bc'] = df_join.bc_count * .8
df_join['weighted_c'] = df_join.c_count * .75

In [45]:
#adding the grades together
sum_grades = (df_join.weighted_a + 
              df_join.weighted_ab + 
              df_join.weighted_b + 
              df_join.weighted_bc + 
              df_join.weighted_c)

df_join['sum_of_grades'] = sum_grades

In [46]:
#adding the student counts together
student_counts = (df_join.a_count +
                  df_join.ab_count +
                  df_join.b_count +
                  df_join.bc_count +
                  df_join.c_count)

df_join['student_count'] = student_counts

In [47]:
#make a ratio of mean grade per class
df_join['grade_ratio'] = df_join.sum_of_grades/ df_join.student_count

#make ratio grade for each student
df_STEM = df_join.loc[df_join.sub_j_name.isin(STEM)]
STEM_grade = df_STEM.grade_ratio.sum()/df_STEM.grade_ratio.count()

df_humanites = df_join.loc[df_join.sub_j_name.isin(humanities)]
humanites_grade = df_humanites.grade_ratio.sum()/df_humanites.grade_ratio.count()

In [48]:
df_STEM.head()

,subject_code,sub_j_name,a_count,ab_count,b_count,bc_count,c_count,start_time,end_time,year,weighted_a,weighted_ab,weighted_b,weighted_bc,weighted_c,sum_of_grades,student_count,grade_ratio
0,220,Chemical and Biological Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.7,0.0,0.75,132.2,140,0.944286
1,320,Electrical and Computer Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.7,0.0,0.75,132.2,140,0.944286
2,346,Engineering Mechanics and Astronautics,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.7,0.0,0.75,132.2,140,0.944286
3,612,Mechanical Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.7,0.0,0.75,132.2,140,0.944286
4,207,Biomedical Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.7,0.0,0.75,132.2,140,0.944286


In [23]:
df_humanites.head()

,subject_code,sub_j_name,a_count,ab_count,b_count,bc_count,c_count,year,weighted_a,weighted_ab,weighted_b,weighted_bc,weighted_c,sum_of_grades,student_count,grade_ratio
26,120,Life Sciences Communication,0,0,0,0,0,2014,0.0,0.0,0.00,0.0,0.0,0.00,0,NaN
40,741,Dance,38,2,0,0,0,2007,36.1,1.8,0.00,0.0,0.0,37.90,40,0.947500
41,741,Dance,18,1,0,0,0,2015,17.1,0.9,0.00,0.0,0.0,18.00,19,0.947368
42,741,Dance,22,3,5,0,0,2015,20.9,2.7,4.25,0.0,0.0,27.85,30,0.928333
43,741,Dance,18,12,0,0,0,2015,17.1,10.8,0.00,0.0,0.0,27.90,30,0.930000


do STEM classes have worse mean grades than humanities?

In [24]:
#find statistics for a hypothesis test
STEM_mean = np.mean(df_STEM.grade_ratio)
STEM_std = np.std(df_STEM.grade_ratio)

humanities_mean = np.mean(df_humanites.grade_ratio)
humanities_mean


z_score = (humanities_mean - STEM_mean)/ STEM_std

stats.norm.sf(z_score)

0.37248820461361276

performance across a decade

In [25]:
STEM_twenty_o_seven_df = df_STEM.loc[df_STEM.year == '2007']

STEM_twenty_seventeen_df = df_STEM.loc[df_STEM.year == '2017']

STEM_o_seven_mean_grade = (STEM_twenty_o_seven_df.grade_ratio.sum()/
                          STEM_twenty_o_seven_df.grade_ratio.count())

STEM_seventeen_mean_grade = (STEM_twenty_seventeen_df.grade_ratio.sum()/
                             STEM_twenty_seventeen_df.grade_ratio.count())

STEM_o_seven_std = np.std(STEM_twenty_o_seven_df.grade_ratio)

STEM_z_score = ((STEM_seventeen_mean_grade - STEM_o_seven_mean_grade)/
                STEM_o_seven_std)

stats.norm.sf(STEM_z_score)

0.44707758931252584

In [26]:
humanites_twenty_o_seven_df = df_humanites.loc[df_humanites.year == '2007']

humanites_twenty_seventeen_df = df_humanites.loc[df_humanites.year == '2017']

humanites_o_seven_mean_grade = (humanites_twenty_o_seven_df.grade_ratio.sum()/
                                humanites_twenty_o_seven_df.grade_ratio.count())

humanites_seventeen_mean_grade = (humanites_twenty_seventeen_df.grade_ratio.sum()/
                                  humanites_twenty_seventeen_df.grade_ratio.count())

humanites_o_seven_std = np.std(humanites_twenty_o_seven_df.grade_ratio)

humanites_z_score = ((humanites_seventeen_mean_grade -
                      humanites_o_seven_mean_grade)/
                     humanites_o_seven_std)

stats.norm.sf(humanites_z_score)

0.39858115440723163

In [49]:
df_join

,subject_code,sub_j_name,a_count,ab_count,b_count,bc_count,c_count,start_time,end_time,year,weighted_a,weighted_ab,weighted_b,weighted_bc,weighted_c,sum_of_grades,student_count,grade_ratio
0,220,Chemical and Biological Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
1,320,Electrical and Computer Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
2,346,Engineering Mechanics and Astronautics,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
3,612,Mechanical Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
4,207,Biomedical Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
5,636,Materials Science and Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
6,490,Industrial and Systems Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
7,418,Geological Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
8,240,Civil and Environmental Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
9,220,Chemical and Biological Engineering,129,8,2,0,1,-1,-1,2015,122.55,7.2,1.70,0.0,0.75,132.20,140,0.944286
